## Importing dependencies

In [66]:
import json
import asyncio
import aiohttp
import time
import os

import pandas as pd
import numpy as np

from setup_nyc_payroll import download_data, read_from_file

## Reading payroll data

In [67]:
FILENAME = 'nyc_payroll.json'
payroll_data = read_from_file(FILENAME)

## Filtering out independent variables


In [68]:
# We are using work_location, regular_hours, and regular_gross_paid to predict title_description

df = pd.DataFrame(data=payroll_data)
dropped_columns = ['fiscal_year', 'agency_name', 'last_name', 'first_name', 'mid_init', 'payroll_number', 'pay_basis', 'leave_status_as_of_july_31', 'ot_hours', 'total_ot_paid', 'total_other_pay', 'base_salary']
df = df.drop(dropped_columns, axis=1)

# Convert columns to numeric and removes negative gross paid
df[["regular_hours", "regular_gross_paid"]] = df[["regular_hours", "regular_gross_paid"]].apply(pd.to_numeric)
df = df[df.agency_start_date != "9999-12-31T00:00:00.000"]
df = df[(df[["regular_gross_paid"]] > 0).all(1)]
df = df.dropna() 

## Days Worked feature

In [69]:
# TODO: Using agency_start_date to make a feature calculating how long they worked in that position
# However for now dropping that feature
df = df.drop(["agency_start_date"], axis=1)

## Setting up X and Y variables

In [70]:
# Setting up X and Y
X = df.drop(["title_description"], axis=1)
Y = df.title_description


# Encoding categorical data

In [ ]:
# We have two categorical data the work_location and title_description